# Learning plurality voting with DeepSets

## Dataset construction

In [32]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch_geometric.nn import MLP, DeepSetsAggregation, conv
from tqdm import tqdm

from geometric_governance.util import RangeOrValue, get_value
from geometric_governance.data import generate_synthetic_election

In [58]:
NUM_VOTERS_RANGE = (3, 50)
NUM_COLLUDERS_RANGE = (0, 20)
NUM_CANDIDATES = 3
TRAIN_BATCH_SIZE = 128
TRAIN_NUM_EPOCHS = 10_000

In [25]:
def fully_connected_directed_edge_index(n):
    row, col = torch.meshgrid(torch.arange(n), torch.arange(n), indexing='ij')
    edge_index = torch.stack([row.flatten(), col.flatten()], dim=0)
    return edge_index[:, edge_index[0] != edge_index[1]]  # Remove self-loops

print(fully_connected_directed_edge_index(5))

tensor([[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
        [1, 2, 3, 4, 0, 2, 3, 4, 0, 1, 3, 4, 0, 1, 2, 4, 0, 1, 2, 3]])


In [70]:
class CollusionModel(nn.Module):
    def __init__(self, num_candidates: int, embedding_size: int = 128):
        super().__init__()
        self.embedding_size = embedding_size

        self.fc1 = nn.Linear(num_candidates, embedding_size)
        self.convs = nn.ModuleList([conv.GATv2Conv(in_channels=(2 * embedding_size), out_channels=(embedding_size)) for _ in range(3)])
        self.fc2 = nn.Linear(embedding_size, num_candidates)

    def forward(self, x, index):
        x = self.fc1(x)
        
        for conv_layer in self.convs:
            # Use Randomized Normal Features to disambiguate voters
            rnf = torch.normal(mean=torch.zeros(x.size(-2), self.embedding_size), std=1.)
            x = torch.concat([x, rnf], dim=-1)

            x = conv_layer(x, index)
            
        x = F.relu(x)
        
        x = self.fc2(x)
        
        x = F.softmax(x, dim=-1)
        
        return x

In [79]:
collusion_model = CollusionModel(NUM_CANDIDATES)
collusion_model.train()
optim = torch.optim.Adam(collusion_model.parameters())

In [81]:
rng = np.random.default_rng(seed=42)

epochs = tqdm(range(TRAIN_NUM_EPOCHS))
for epoch in epochs:
    num_voters = get_value(NUM_VOTERS_RANGE, rng)
    num_colluders = get_value(NUM_COLLUDERS_RANGE, rng)
    voter_utilities = rng.dirichlet(
        alpha=(1.,) * NUM_CANDIDATES, size=(num_voters + 1)
    )
    # Duplicate the last candidate's data to generate colluding voters' data
    # so that all colluding candidates share the same utility
    collusion_utility = voter_utilities[-1][None, :]
    collusion_candidate = np.argmax(collusion_utility)
    collusion_utilities = np.tile(collusion_utility, (num_colluders, 1))
    collusion_utilities = torch.from_numpy(collusion_utilities).float()
    #print(collusion_utilities)

    # Colluders cast their vote jointly (by adjusting the utilities)
    collusion_utilities = collusion_model(collusion_utilities, fully_connected_directed_edge_index(len(collusion_utilities)))
    #print(collusion_utilities)

    # Assimilate votes
    voter_utilities = torch.cat([torch.from_numpy(voter_utilities[:-1]).float(), collusion_utilities])

    # Plurality voting
    election_result = F.softmax(torch.sum(voter_utilities, dim=-2), dim=-1)
    #print(election_result)

    # The colluders' loss is how close they can get to getting their chosen candidate to win the election/lottery
    ideal_result = torch.zeros_like(election_result)
    ideal_result[collusion_candidate] = 1.
    #print(ideal_result)

    optim.zero_grad()
    collusion_loss = torch.nn.functional.cross_entropy(election_result, ideal_result)
    collusion_loss.backward()
    optim.step()

    epochs.set_postfix({"collusion_loss": collusion_loss})
    #break
    


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:28<00:00, 347.13it/s, collusion_loss=tensor(1.5514, grad_fn=<DivBackward1>)]


In [82]:
collusion_utilities

tensor([[0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.]], grad_fn=<SoftmaxBackward0>)

In [83]:
election_result

tensor([6.6456e-09, 1.0000e+00, 1.7945e-08], grad_fn=<SoftmaxBackward0>)